In [1]:
import json

import requests


RULM_URL = 'https://api.rulm.alexkuk.ru/v1'


def rulm_models():
    return requests.get(f'{RULM_URL}/models').json()


def rulm_complete_stream(prompt, model='saiga-7b-q4', max_tokens=128, temperature=0.2):
    response = requests.post(
        f'{RULM_URL}/complete',
        json={
            'prompt': prompt,
            'model': model,
            'max_tokens': max_tokens,
            'temperature': temperature
        },
        stream=True
    )
    for line in response.iter_lines():
        yield json.loads(line)


# {'n_past': 0, 'n_tokens': 78, 'text': None}
# {'n_past': 16, 'n_tokens': 78, 'text': None}
# {'n_past': 32, 'n_tokens': 78, 'text': None}
# {'n_past': 48, 'n_tokens': 78, 'text': None}
# {'n_past': 64, 'n_tokens': 78, 'text': None}
# {'n_past': 78, 'n_tokens': 78, 'text': None}
# {'n_past': None, 'n_tokens': None, 'text': 'В'}
# {'n_past': None, 'n_tokens': None, 'text': 'ы'}
# {'n_past': None, 'n_tokens': None, 'text': 'ход'}
# {'n_past': None, 'n_tokens': None, 'text': 'ные'}
# {'n_past': None, 'n_tokens': None, 'text': ' дан'}


def show_rulm_stream(items):
    parts = []
    for item in items:
        if item['text']:
            parts.append(item['text'])
            print(item['text'], flush=True, end='')
        else:
            percent = item['n_past'] / item['n_tokens'] * 100
            print(f'{percent:.0f}%', flush=True, end=' ')
            if percent == 100:
                print('\n', flush=True)
    return ''.join(parts)


def rulm_complete(prompt, **kwargs):
    items = rulm_complete_stream(prompt, **kwargs)
    parts = []
    for item in items:
        if item['text']:
            parts.append(item['text'])
    return ''.join(parts)

In [2]:
prompt = '9 мая 2023 года это какой день недели?'
model = 'saiga-7b-q4'

items = rulm_complete_stream(prompt, model, max_tokens=64, temperature=0)
output = show_rulm_stream(items)

0% 20% 41% 61% 81% 100% 

9 мая 2023 года будет вторник. <end>

In [3]:
output = rulm_complete(prompt, model=model, max_tokens=64, temperature=0)
print(output)

9 мая 2023 года будет вторник. <end>


In [4]:
for model in rulm_models():
    print(model)
    print(rulm_complete(prompt, model=model, max_tokens=64, temperature=0))
    print()

ru-alpaca-7b-f16
 9 мая 2023 года - вторник.

ru-alpaca-7b-q4
 9 мая 2023 года будет вторник.

saiga-7b-f16
9 мая 2023 года будет вторник. Это второй день недели в году. <end>

saiga-7b-q4
9 мая 2023 года будет вторник. <end>

saiga-7b-v2-f16
9 мая 2023 года будет вторник.</s>

saiga-7b-v2-q4
Выходные данные: 52-й день года.</s>

